In [19]:
import torch
from torch import optim
from torch.nn import functional as F
from torch import nn

import dlc_practical_prologue as prologue

In [20]:
def compute_nb_errors(model, data_input, data_target, mini_batch_size):

    nb_data_errors = 0

    for b in range(0, data_input.size(0), mini_batch_size):
        output = model(data_input.narrow(0, b, mini_batch_size))
        _, predicted_classes = torch.max(output, 1)
        for k in range(mini_batch_size):
            if data_target[b + k] != predicted_classes[k]:
                nb_data_errors = nb_data_errors + 1

    return nb_data_errors

In [21]:
def train_model(model, train_input, train_target, nb_epochs, mini_batch_size):
    criterion = nn.CrossEntropyLoss()
    eta = 1e-3
    optimizer = optim.Adam(model.parameters(), lr = eta)

    for e in range(nb_epochs):
        acc_loss = 0
        
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            acc_loss = acc_loss + loss.item()
            model.zero_grad()
            loss.backward()
            optimizer.step()
    
        #print(e, acc_loss)

In [22]:
class MultilayerPerceptron(nn.Module):
    def __init__(self):
        super(MultilayerPerceptron, self).__init__()
        
        nb_hidden = 100
        input_size = 2*14*14
        
        self.layers = nn.Sequential(
            nn.Linear(input_size, nb_hidden),
            nn.ReLU(),
            nn.Linear(nb_hidden, nb_hidden),
            nn.Linear(nb_hidden, nb_hidden),
            nn.Linear(nb_hidden, 2),
            nn.LogSoftmax(dim=1)
        )
        
    def forward(self, x):
        x = x.view(x.size(0),-1)
        out = self.layers(x)
    
        return out

In [23]:
train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(1000)
    
# train_input, train_target, train_classes \
#     = Variable(train_input), Variable(train_target), Variable(train_classes)
# test_input, test_target, test_classes \
#     = Variable(test_input), Variable(test_target), Variable(test_classes)

def get_tests(n):
    M = []
    for k in range (0, n):
        L = []
        _, _, _, test_input, test_target, test_classes =  prologue.generate_pair_sets(1000)
        L.append(test_input)
        L.append(test_target)
        L.append(test_classes)
        M.append(L)
    return M

MLP = MultilayerPerceptron()
nb_epochs = 100
mini_batch_size = 100

train_model(MLP, train_input, train_target, nb_epochs, mini_batch_size)

nb_train_errors = compute_nb_errors(MLP, train_input, train_target, mini_batch_size)
print('train error MLP with weight sharing and with auxiliary loss {:0.2f}% {:f}/{:f}'.format((100 * nb_train_errors) / train_input.size(0), nb_train_errors, train_target.size(0)))

L = get_tests(10)
average_nb_test_error = 0
for k in range (0, len(L)):
    nb_test_errors = compute_nb_errors(MLP, L[k][0], L[k][1], mini_batch_size)
    average_nb_test_error += nb_test_errors
    print('test error MultilayerPerceptron {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / L[k][0].size(0),
                                                nb_test_errors, L[k][0].size(0)))
print('Average test error MultilayerPerceptron {:0.2f}% {:0.1f}/{:d}'.format((100*average_nb_test_error/10) / L[0][0].size(0),
                                                                  average_nb_test_error/len(L), L[0][0].size(0) ))

train error MLP with weight sharing and with auxiliary loss 0.00% 0.000000/1000.000000
test error MultilayerPerceptron 22.40% 224/1000
test error MultilayerPerceptron 24.10% 241/1000
test error MultilayerPerceptron 22.40% 224/1000
test error MultilayerPerceptron 21.60% 216/1000
test error MultilayerPerceptron 23.60% 236/1000
test error MultilayerPerceptron 22.20% 222/1000
test error MultilayerPerceptron 22.30% 223/1000
test error MultilayerPerceptron 24.50% 245/1000
test error MultilayerPerceptron 23.50% 235/1000
test error MultilayerPerceptron 21.20% 212/1000
Average test error MultilayerPerceptron 22.78% 227.8/1000
